# Dogs vs Cats Classification Using SVM

In [23]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
from libsvm.svmutil import *
from libsvm.svm import *
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

## 1. Load Dataset

In [2]:
# load training and test set
y_train, X_train = svm_read_problem('./DogsVsCats/DogsVsCats.train')
y_test, X_test = svm_read_problem('./DogsVsCats/DogsVsCats.test')

# convert to numpy.ndarray
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

## 2. Linear Kernel and Polynomial Kernel CV

In [28]:
linear_params = '-t 0 -b 0'  # linear SVM and use it for classification
poly_params = '-t 1 -d 5 -b 0'  # polynomial SVM with degree of 5 and use it for classification

# K-fold cross-validation
kf = KFold(n_splits=10, shuffle=True)
linear_val_accuracies = []; poly_val_accuracies = []
iters = 1

for train_idx, val_idx in kf.split(X_train):
    
    # split dataset
    X_train_i, X_val_i = X_train[train_idx], X_train[val_idx]
    y_train_i, y_val_i = y_train[train_idx], y_train[val_idx]
    
    # train linear SVM
    print("Training Linear SVM: {}/{}".format(iters, kf.get_n_splits(X_train)))
    linear_model = svm_train(y_train_i, X_train_i, linear_params)
    print("Training Linear SVM Done: {}/{}".format(iters, kf.get_n_splits(X_train)))
    
    # validate linear SVM
    print("Testing Linear SVM: {}/{}".format(iters, kf.get_n_splits(X_train)))
    _, val_acc, _ = svm_predict(y_val_i, X_val_i, linear_model)
    linear_val_accuracies.append(val_acc[0])
    print("Testing Linear SVM Done: {}/{}".format(iters, kf.get_n_splits(X_train)))
    
    # train polynomial SVM
    print("Training Polynomial SVM: {}/{}".format(iters, kf.get_n_splits(X_train)))
    poly_model = svm_train(y_train_i, X_train_i, poly_params)
    print("Training Polynomial SVM Done: {}/{}".format(iters, kf.get_n_splits(X_train)))
    
    # validate polynomial SVM
    print("Testing Polynomial SVM: {}/{}".format(iters, kf.get_n_splits(X_train)))
    _, val_acc, _ = svm_predict(y_val_i, X_val_i, poly_model)
    poly_val_accuracies.append(val_acc[0])
    print("Testing Polynomial SVM Done: {}/{}".format(iters, kf.get_n_splits(X_train)))
    
    iters += 1

Training Linear SVM: 1/10
Training Linear SVM Done: 1/10
Testing Linear SVM: 1/10
Accuracy = 59.84% (748/1250) (classification)
Testing Linear SVM Done: 1/10
Training Polynomial SVM: 1/10
Training Polynomial SVM Done: 1/10
Testing Polynomial SVM: 1/10
Accuracy = 49.6% (620/1250) (classification)
Testing Polynomial SVM Done: 1/10
Training Linear SVM: 2/10
Training Linear SVM Done: 2/10
Testing Linear SVM: 2/10
Accuracy = 57.76% (722/1250) (classification)
Testing Linear SVM Done: 2/10
Training Polynomial SVM: 2/10
Training Polynomial SVM Done: 2/10
Testing Polynomial SVM: 2/10
Accuracy = 49.6% (620/1250) (classification)
Testing Polynomial SVM Done: 2/10
Training Linear SVM: 3/10
Training Linear SVM Done: 3/10
Testing Linear SVM: 3/10
Accuracy = 61.28% (766/1250) (classification)
Testing Linear SVM Done: 3/10
Training Polynomial SVM: 3/10
Training Polynomial SVM Done: 3/10
Testing Polynomial SVM: 3/10
Accuracy = 48.8% (610/1250) (classification)
Testing Polynomial SVM Done: 3/10
Trainin

In [32]:
print("Average Validation Accuracy of Linear SVM: %.2f %%" % (np.mean(linear_val_accuracies)))
print("Average Validation Accuracy of Polynomial SVM: %.2f %%" % (np.mean(poly_val_accuracies)))

Average Validation Accuracy of Linear SVM: 59.86 %
Average Validation Accuracy of Polynomial SVM: 49.15 %


## 3. Train Entire Dataset

In [37]:
# Linear SVM

# train
print("Training Linear SVM:")
linear_model = svm_train(y_train, X_train, linear_params)
print("Training Linear SVM Done")
_, train_acc, _ = svm_predict(y_train, X_train, linear_model)
print("Training Accuracy of Linear SVM: %.2f %%" % train_acc[0])

# test
print("Testing Linear SVM:")
_, test_acc, _ = svm_predict(y_test, X_test, linear_model)
print("Testing Linear SVM Done")
print("Test Accuracy of Linear SVM: %.2f %%" % test_acc[0])

Training Linear SVM:
Training Linear SVM Done
Accuracy = 60.12% (7515/12500) (classification)
Training Accuracy of Linear SVM: 60.12 %
Testing Linear SVM
Accuracy = 59.2% (7400/12500) (classification)
Testing Linear SVM Done
Test Accuracy of Linear SVM: 59.20 %


In [38]:
# Polynomial SVM

# train
print("Training Polynomial SVM:")
poly_model = svm_train(y_train, X_train, poly_params)
print("Training Polynomial SVM Done")
_, train_acc, _ = svm_predict(y_train, X_train, poly_model)
print("Training Accuracy of Polynomial SVM: %.2f %%" % train_acc[0])

# test
print("Testing Polynomial SVM")
_, test_acc, _ = svm_predict(y_test, X_test, poly_model)
print("Testing Polynomial SVM Done")
print("Test Accuracy of Polynomial SVM: %.2f %%" % test_acc[0])

Training Polynomial SVM:
Training Polynomial SVM Done
Accuracy = 50.024% (6253/12500) (classification)
Training Accuracy of Polynomial SVM: 50.02 %
Testing Polynomial SVM
Accuracy = 50.048% (6256/12500) (classification)
Testing Polynomial SVM Done
Test Accuracy of Polynomial SVM: 50.05 %


Polynomial kernel has higher test accuracy.

## 4. Boosting

In [41]:
# K = 10
adaboost = AdaBoostClassifier(base_estimator=SVC(probability=True, kernel='linear'), n_estimators=10)
adaboost.fit(np.array([list(x.values()) for x in X_train]), y_train)
y_pred = adaboost.predict([list(x.values()) for x in X_test])
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with Boosting Iterations of 10: %.2f %%" % (100*accuracy))

Test Accuracy with Boosting Iterations of 10: 56.41 %


The result show the accuracy of boosted SVM is lower than single SVM, which means the Adaboost SVM does not improve the performance. The main reason I reckon is SVM is a strong classifier, while Adaboost prefers multiple weak classifiers, so that is the reason why decision stump is used as default base estimator in AdaBoostClassifier.

In [42]:
# K = 20
adaboost = AdaBoostClassifier(base_estimator=SVC(probability=True, kernel='linear'), n_estimators=20)
adaboost.fit(np.array([list(x.values()) for x in X_train]), y_train)
y_pred = adaboost.predict([list(x.values()) for x in X_test])
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with Boosting Iterations of 20: %.2f %%" % (100*accuracy))

Test Accuracy with Boosting Iterations of 20: 54.70 %


When the number of boosting iterations increases to 20, the test accuracy is getting worse than K = 10. The reason is the same: Adaboost prefers to weak classifiers, but SVM is a strong classifier, so the performance will be even worse, even through we increase the number of boosting iterations. We compare the performance of Adaboost using SVM and decision stump as base estimator, respectively. We find the using decision stump is better than SVM, and with the increase of boosting iterations, the performance also improves. The results are shown below

In [66]:
# K = 10

# Using decision stump as base estimator
adaboost = AdaBoostClassifier(n_estimators=10)
adaboost.fit(np.array([list(x.values()) for x in X_train]), y_train)
y_pred = adaboost.predict([list(x.values()) for x in X_test])
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with Boosting Iterations of 10 Using Decision Stump: %.2f %%" % (100*accuracy))

Test Accuracy with Boosting Iterations of 10 Using Decision Stump: 62.73 %


In [68]:
# K = 20

# Using decision stump as base estimator
adaboost = AdaBoostClassifier(n_estimators=20)
adaboost.fit(np.array([list(x.values()) for x in X_train]), y_train)
y_pred = adaboost.predict([list(x.values()) for x in X_test])
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy with Boosting Iterations of 20 Using Decision Stump: %.2f %%" % (100*accuracy))

Test Accuracy with Boosting Iterations of 20 Using Decision Stump: 64.42 %


## 5. Conclusion

In our experiment, we did cross validation for model selection between linear SVM and polynomial SVM, and the results show that the averaged test accuracy of linear SVM is 60.12%, which outperforms polynomial SVM. Thus, we choosed polynomial SVM as base classifier to do Adaboost. However, the boosted result is not what we expected that boosted accuracy will be larger than non-boosted accuracy. The underlying reason is we need to choose weak classifier as our base estimator, and we also did an ablation study where we compared the performace of Adaboost with the same hyperparameters but using decision stump as base estimator. The result confirmed that using weak classifiers would increase the overall performance, and if we increase the number of boosting iterations, the test accuracy will also increase.